# 02_limpieza.ipynb — PROYECTO AURORA

In [89]:
import pandas as pd
import numpy as np
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 120)

RAW_PATH = "../data/raw/"
CLEAN_PATH = "../data/clean/"

if not os.path.exists(CLEAN_PATH):
    os.makedirs(CLEAN_PATH)

print("Rutas de trabajo:")
print("RAW_PATH   →", RAW_PATH)
print("CLEAN_PATH →", CLEAN_PATH)

Rutas de trabajo:
RAW_PATH   → ../data/raw/
CLEAN_PATH → ../data/clean/


In [90]:
def anonimizar_game(df, alias_juego="Proyecto Aurora"):
    """
    Si el DataFrame tiene una columna 'Game', la renombra a 'Juego'
    y fuerza el valor alias_juego en todas las filas.
    Si no la tiene, simplemente añade la columna 'Juego' con el alias.
    """
    if "Game" in df.columns:
        df = df.rename(columns={"Game": "Juego"})
    if "Juego" not in df.columns:
        df["Juego"] = alias_juego
    else:
        df["Juego"] = alias_juego
    return df


In [91]:
def leer_raw(nombre, sep=",", encoding="utf-8"):
    ruta = os.path.join(RAW_PATH, nombre)
    try:
        df = pd.read_csv(ruta, sep=sep, encoding=encoding)
        print(f"✅ Cargado RAW: {nombre} | Filas: {df.shape[0]} | Columnas: {df.shape[1]}")
        return df
    except Exception as e:
        print(f"❌ Error al leer {nombre}: {e}")
        return None


In [92]:
downloads_steam = leer_raw("downloads_steam.csv")
steam_userstats = leer_raw("SteamUserStats.csv")
steam_sales = leer_raw("SteamInGameSales.csv")
steam_hourly = leer_raw("SteamHourlyConcurrents.csv")
canalB_raw = leer_raw("reportNTK22-25.csv")
downloads_region = leer_raw("DownloadsByRegion.csv")
playfab_raw = leer_raw("playfab.csv")


✅ Cargado RAW: downloads_steam.csv | Filas: 1274 | Columnas: 2
✅ Cargado RAW: SteamUserStats.csv | Filas: 1274 | Columnas: 3
✅ Cargado RAW: SteamInGameSales.csv | Filas: 9403 | Columnas: 18
✅ Cargado RAW: SteamHourlyConcurrents.csv | Filas: 30550 | Columnas: 4
✅ Cargado RAW: reportNTK22-25.csv | Filas: 3834 | Columnas: 17
✅ Cargado RAW: DownloadsByRegion.csv | Filas: 194 | Columnas: 3
✅ Cargado RAW: playfab.csv | Filas: 39127 | Columnas: 12


In [93]:

steam_dl = downloads_steam.copy()

steam_dl = steam_dl.rename(columns={
    "Date": "Fecha",
    "Units": "Descargas"
})

steam_dl["Fecha"] = pd.to_datetime(
    steam_dl["Fecha"],
    errors="coerce"       
)

steam_dl = anonimizar_game(steam_dl)

steam_dl.to_csv(os.path.join(CLEAN_PATH, "steam_descargas_clean.csv"), index=False)

steam_dl.head()


C:\Users\cubea\AppData\Local\Temp\ipykernel_41752\3934875399.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  steam_dl["Fecha"] = pd.to_datetime(


,Fecha,Descargas,Juego
0,2022-01-06,153,Proyecto Aurora
1,2022-02-06,149,Proyecto Aurora
2,2022-03-06,193,Proyecto Aurora
3,2022-04-06,169,Proyecto Aurora
4,2022-05-06,205,Proyecto Aurora


# LIMPIEZA STEAM USER STATS (DAU + CONCURRENCIA)

In [94]:
steam_stats = steam_userstats.copy()

steam_stats = steam_stats.rename(columns={
    "DateReported": "Fecha",
    "DailyActiveUsers": "DAU",
    "PeakConcurrentUsers": "Usuarios_Concurrentes_Pico"
})

steam_stats["Fecha"] = pd.to_datetime(steam_stats["Fecha"], errors="coerce")

steam_stats = anonimizar_game(steam_stats)  # ← AQUI

steam_stats.to_csv(os.path.join(CLEAN_PATH, "steam_stats_clean.csv"), index=False)

steam_stats.head()



,Fecha,DAU,Usuarios_Concurrentes_Pico,Juego
0,2022-06-01,1294,83,Proyecto Aurora
1,2022-06-02,1315,80,Proyecto Aurora
2,2022-06-03,1355,87,Proyecto Aurora
3,2022-06-04,1307,86,Proyecto Aurora
4,2022-06-05,1378,96,Proyecto Aurora


In [95]:
steam_sales_clean = steam_sales.copy()

# 0) Renombrar Date -> Fecha
steam_sales_clean = steam_sales_clean.rename(columns={
    "Date": "Fecha"
})

# 1) Convertir fecha
steam_sales_clean["Fecha"] = pd.to_datetime(
    steam_sales_clean["Fecha"],
    errors="coerce"
)

# 2) Eliminar columnas sensibles / que no necesitamos para el fact
cols_a_eliminar = [
    "Product(ID#)",
    "Item(ID#)",
    "Description",
    "Category",
    "Country Code",
    "Country",
    "Region",
    "Currency"
]
steam_sales_clean = steam_sales_clean.drop(
    columns=[c for c in cols_a_eliminar if c in steam_sales_clean.columns],
    errors="ignore"
)

# 3) Renombrar columnas numéricas a castellano
steam_sales_clean = steam_sales_clean.rename(columns={
    "Gross Units Sold": "Unidades_Vendidas_Brutas",
    "Chargeback/Returns": "Devoluciones_Unidades",
    "Net Units Sold": "Unidades_Vendidas",
    " Average Price": "Precio_Medio_USD",   # ojo al espacio inicial
    "Gross Steam Sales (USD)": "Ingresos_Brutos_USD",
    "Chargeback/Returns (USD)": "Devoluciones_USD",
    "VAT/Tax (USD)": "Impuestos_USD",
    "Net Steam Sales (USD)": "Ingresos_USD"
})

# 4) Agregar por fecha (sumar ventas de todos los productos del día)
agg_dict = {
    "Unidades_Vendidas_Brutas": "sum",
    "Devoluciones_Unidades": "sum",
    "Unidades_Vendidas": "sum",
    "Precio_Medio_USD": "mean",          # media del precio medio diario
    "Ingresos_Brutos_USD": "sum",
    "Devoluciones_USD": "sum",
    "Impuestos_USD": "sum",
    "Ingresos_USD": "sum"
}

steam_sales_clean = (
    steam_sales_clean
    .groupby("Fecha", as_index=False)
    .agg(agg_dict)
)

# 5) Asegurarnos de que no queda 'Juego' aquí
if "Juego" in steam_sales_clean.columns:
    steam_sales_clean = steam_sales_clean.drop(columns=["Juego"])

# 6) Guardar limpio
steam_sales_clean.to_csv(os.path.join(CLEAN_PATH, "steam_ventas_clean.csv"), index=False)

steam_sales_clean.head()

,Fecha,Unidades_Vendidas_Brutas,Devoluciones_Unidades,Unidades_Vendidas,Precio_Medio_USD,Ingresos_Brutos_USD,Devoluciones_USD,Impuestos_USD,Ingresos_USD
0,2022-06-01,44,0,44,8.913636,449.68,0.0,-21.24,428.44
1,2022-06-02,67,0,67,20.268750,1294.24,0.0,-98.28,1195.96
2,2022-06-03,117,0,117,21.067500,2304.43,0.0,-174.96,2129.47
3,2022-06-04,83,0,83,15.625000,1031.35,0.0,-89.98,941.37
4,2022-06-05,136,0,136,12.862727,1732.55,0.0,-140.89,1591.66


In [96]:
steam_hourly_clean = steam_hourly.copy()

# Detectar la columna de fecha/hora
col = [c for c in steam_hourly.columns if "hour" in c.lower()][0]

steam_hourly_clean = steam_hourly_clean.rename(columns={
    col: "FechaHora"
})

steam_hourly_clean["FechaHora"] = pd.to_datetime(steam_hourly_clean["FechaHora"], errors="coerce")

# Extraer solo día 
steam_hourly_clean["Fecha"] = steam_hourly_clean["FechaHora"].dt.date

steam_hourly_clean.to_csv(os.path.join(CLEAN_PATH, "steam_concurrencia_horaria_clean.csv"), index=False)

steam_hourly_clean.head()


,FechaHora,PeakConcurrentUsers,PeakConcurrentUsersMac,PeakConcurrentUsersLinux,Fecha
0,2022-06-01 00:00:00,58,0,1,2022-06-01
1,2022-06-01 01:00:00,59,0,0,2022-06-01
2,2022-06-01 02:00:00,54,0,0,2022-06-01
3,2022-06-01 03:00:00,53,0,0,2022-06-01
4,2022-06-01 04:00:00,54,0,1,2022-06-01


In [97]:
canalB = canalB_raw.copy()
canalB = canalB[canalB["Device"] == "TOTAL"].copy()

canalB = canalB.rename(columns={
    "Date": "Fecha",
    "Download": "Descargas",
    "DAU": "DAU",
    "Gold Spent": "Oro_Gastado",
    "ARPDAU": "ARPDAU",
    "ARPU": "ARPU",
    "ARPPU": "ARPPU"
})

canalB["Fecha"] = pd.to_datetime(canalB["Fecha"], errors="coerce")

# 👇 ESTA YA LA TENÍAS: ingresos en USD
canalB["Ingresos_USD"] = canalB["Oro_Gastado"] / 100.0

# 👇 AÑADIR ESTO: oro convertido a USD como métrica propia
canalB["Oro_Gastado_USD"] = canalB["Oro_Gastado"] / 100.0

canalB = anonimizar_game(canalB)

canalB.to_csv(os.path.join(CLEAN_PATH, "canalB_clean.csv"), index=False)


In [98]:
regiones = downloads_region.copy()

# Este archivo no entra en el fact diario.
# Se limpia solo para uso de Tableau en top regiones/países.

regiones = regiones.rename(columns={
    "Region": "Region",
    "Total Downloads": "Descargas_Totales",
    "Share": "Porcentaje"
})

regiones.to_csv(os.path.join(CLEAN_PATH, "steam_regiones_clean.csv"), index=False)
regiones.head()


,Region,Descargas_Totales,Porcentaje
0,North America,35292,45.4%
1,Western Europe,11351,14.6%
2,Latin America,8348,10.7%
3,Central Asia,6266,8.0%
4,Asia,4929,6.3%


In [99]:

playfab = playfab_raw.copy()

# Convertir fecha correctamente
playfab["Fecha"] = pd.to_datetime(playfab["Date"], errors="coerce")

# Eliminar datos sensibles
cols_sensibles = ["PlayerID", "Item ID", "Server", "Type", "Subtype", "Unnamed: 3"]
playfab = playfab.drop(columns=[c for c in cols_sensibles if c in playfab.columns], errors="ignore")

# Limpiar campo Revenue si viene como "$13,50"
if "Revenue" in playfab.columns:
    playfab["Revenue"] = (
        playfab["Revenue"]
        .astype(str)
        .str.replace("$", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    playfab["Revenue"] = pd.to_numeric(playfab["Revenue"], errors="coerce")
    
playfab = anonimizar_game(playfab)    

playfab.to_csv(os.path.join(CLEAN_PATH, "playfab_clean.csv"), index=False)

playfab.head()


,Event,Date,Nutaku Gold,DAY,Month,Revenue,Fecha,Juego
0,nc_spent,2024-12-03T17:37:04.837036Z,900.0,3/12/2024,diciembre-2024,9.0,2024-12-03 17:37:04.837036+00:00,Proyecto Aurora
1,nc_spent,2024-12-03T17:41:12.862961Z,1000.0,3/12/2024,diciembre-2024,10.0,2024-12-03 17:41:12.862961+00:00,Proyecto Aurora
2,nc_spent,2024-12-03T17:44:55.827691Z,900.0,3/12/2024,diciembre-2024,9.0,2024-12-03 17:44:55.827691+00:00,Proyecto Aurora
3,nc_spent,2024-12-03T18:15:45.686973Z,1350.0,3/12/2024,diciembre-2024,13.5,2024-12-03 18:15:45.686973+00:00,Proyecto Aurora
4,nc_spent,2024-12-03T18:15:54.967943Z,1350.0,3/12/2024,diciembre-2024,13.5,2024-12-03 18:15:54.967943+00:00,Proyecto Aurora


In [100]:
print("""
📌 LIMPIEZA COMPLETADA — PROYECTO AURORA

Datasets generados en data/clean/:

- steam_descargas_clean.csv
- steam_stats_clean.csv
- steam_ventas_clean.csv
- steam_concurrencia_horaria_clean.csv
- canalB_clean.csv  (solo línea TOTAL)
- steam_regiones_clean.csv
- playfab_clean.csv (EDA opcional; NO para el fact diario)

Todo listo para Notebook 03_fact_diario.ipynb
""")



📌 LIMPIEZA COMPLETADA — PROYECTO AURORA

Datasets generados en data/clean/:

- steam_descargas_clean.csv
- steam_stats_clean.csv
- steam_ventas_clean.csv
- steam_concurrencia_horaria_clean.csv
- canalB_clean.csv  (solo línea TOTAL)
- steam_regiones_clean.csv
- playfab_clean.csv (EDA opcional; NO para el fact diario)

Todo listo para Notebook 03_fact_diario.ipynb

